In [1]:
import json
import gzip
from tqdm import tqdm

Брал корпуса электроники, смотрел устройства Samsung Galaxy
1. http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Electronics_5.json.gz
2. https://forms.gle/UEkkJs69e7Z5A5Ps9

In [2]:
def parse(path):
    g = gzip.open(path, 'r')
    for l in g:
        yield eval(l)

In [3]:
data = parse('reviews_Electronics_5.json.gz')

In [4]:
texts = []
for i in tqdm(data):
    texts.append(i['reviewText'])
    if len(texts) > 100000:
        break

100000it [00:05, 18327.24it/s]


In [5]:
from pprint import pprint

data2 = parse('meta_Electronics.json.gz')

forner=[]

for i in data2:
    forner.append(i['title'])
    if len(forner) > 10000:
        break

In [6]:
ids = []
for i in tqdm(data2):
    if i['brand'] == 'Samsung':
        ids.append(i['asin'])

776444it [04:09, 3108.19it/s]


In [7]:
samsung_reviews = []

for i in tqdm(data):
    if i['asin'] in ids:
        samsung_reviews.append(i)

1589187it [04:51, 5455.27it/s]


In [13]:
len(samsung_reviews)

20360

In [ ]:
samsung_reviews = [text['reviewText'] for text in samsung_reviews]

In [16]:
from string import punctuation
punctuation += '...' + '—' + '…' + '«»'

In [17]:
samsung_reviews2 = []

for text in samsung_reviews:
    temp=[]
    for token in text.split():
        temp.append(token.lower().strip(punctuation).strip())
    
    samsung_reviews2.append(' '.join(temp))

In [18]:
samsung_reviews_forw2v = []

for text in samsung_reviews:
    temp=[]
    for token in text.split():
        temp.append(token.lower().strip(punctuation).strip())
    
    samsung_reviews_forw2v.append(temp)

Попробую выделять телефоны Samsung линейкли Galaxy. Посмотрю синонимы слова телефон

In [15]:
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\trekc\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


:(

In [16]:
print(wn.synsets('smartphone'))
print(wn.synsets('phone'))
print(wn.synsets('cell'))
print(wn.synsets('tablet'))

[]
[Synset('telephone.n.01'), Synset('phone.n.02'), Synset('earphone.n.01'), Synset('call.v.03')]
[Synset('cell.n.01'), Synset('cell.n.02'), Synset('cell.n.03'), Synset('cell.n.04'), Synset('cellular_telephone.n.01'), Synset('cell.n.06'), Synset('cell.n.07')]
[Synset('tablet.n.01'), Synset('pad.n.01'), Synset('tablet.n.03'), Synset('pill.n.02')]


In [17]:
import stanza
ners={}

nlp = stanza.Pipeline(lang='en', processors='tokenize,ner')
doc = nlp(' '.join(forner[:10000]))
for sent in doc.sentences:
    for ent in sent.ents:  # достаем NE
        try:
            if ent.text not in ners[ent.type]:
                ners[ent.type].append(ent.text)
        except KeyError:
            ners[ent.type] = []
            ners[ent.type].append(ent.text)

2021-12-04 19:15:36 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | combined  |
| ner       | ontonotes |

2021-12-04 19:15:36 INFO: Use device: gpu
2021-12-04 19:15:36 INFO: Loading: tokenize
2021-12-04 19:15:39 INFO: Loading: ner
2021-12-04 19:15:40 INFO: Done loading processors!


In [18]:
for ner in ners['PRODUCT']:
    if 'samsung galaxy' in ner.lower():
        print(ner.lower())

microsdhc samsung galaxy
samsung galaxy
microsdxc samsung galaxy
samsung galaxy note 4
samsung galaxy tab
samsung galaxy core prime
samsung galaxy tab 2 7.0&quot
samsung galaxy 5
samsung galaxy rush micro sdhc card
samsung galaxy s blaze
samsung galaxy reverb phone
samsung galaxy stellar phone
samsung galaxy rush phone
samsung galaxy precedent phone
samsung galaxy ace plus phone
samsung galaxy s3 smartphone
samsung galaxy rugby pro smartphone
samsung galaxy s3
samsung galaxy s iii
microsdxc samsung galaxy note 3 card
samsung galaxy stratosphere
samsung galaxy grand 2
samsung galaxy note 3
samsung galaxy s5 active
samsung galaxy tab 4
samsung galaxy player 4 phone phone
samsung galaxy prevail phone
samsung galaxy indulge phone phone
samsung galaxy prevail phone phone
samsung galaxy 5 phone phone
samsung galaxy ace phone phone


In [19]:
from gensim.models import Word2Vec

In [20]:
model =  Word2Vec(samsung_reviews_forw2v, min_count=1)

In [21]:
model.wv.most_similar('samsung', topn=10)

[("samsung's", 0.6227081418037415),
 ('panasonic', 0.5476346611976624),
 ('model', 0.5429607033729553),
 ('sammy', 0.5132973194122314),
 ('sharp...???-i', 0.49897024035453796),
 ('sony', 0.4985235333442688),
 ('samsungs', 0.4824788272380829),
 ('6700', 0.4803653359413147),
 ('company', 0.4783826768398285),
 ('pn50c7000', 0.47784656286239624)]

In [22]:
model.wv.most_similar('galaxy', topn=10)

[('10.1', 0.8556718826293945),
 ('s3', 0.8454987406730652),
 ('s4', 0.811606228351593),
 ('s5', 0.7802479863166809),
 ('siii', 0.7718286514282227),
 ('tab', 0.7599642872810364),
 ('8.9', 0.7590399384498596),
 ('8.0', 0.7473368644714355),
 ('s2', 0.7457958459854126),
 ('27.0', 0.7391760945320129)]

In [23]:
model.wv.most_similar('phone', topn=10)

[('smartphone', 0.7459598183631897),
 ('s4', 0.6870135068893433),
 ('s3', 0.6662060618400574),
 ('s5', 0.6637851595878601),
 ('account', 0.6463894248008728),
 ('cellphone', 0.6344552636146545),
 ('phones', 0.6188552379608154),
 ('cell', 0.6054475903511047),
 ('iphone', 0.5884494185447693),
 ('hotspot', 0.5728897452354431)]

In [24]:
tags2 = ['galaxy', 'phone', 'smartphone', 'tablet']
n_tags = []
for tag in tags2:
    w2v = model.wv.most_similar(tag, topn=10)
    for w in w2v:
        n_tags.append(w[0])

Собственно способы реализации:
1. Написать словарь сущностей (я это тоже делал). Дополнитиельные данные не нужны. Минусы: грязный результат, его сложно фильтровать (типо нельзя поставить ограничение по постеггингу, ну либо неудобно). Плюсы: быстро.
2. Написать правила (то, что сделал я). Нужно понимание устройства товарной линейки. Плюсы и минусы: можно получить хороший результат, но медленно работает.
3. Можно воспользоваться автоматическим экстрактером, а дальше отлфилтровать результат по шаблону. Нужно выяснить, что за шаблон нужен. Плюсы: просто и удобно. Минусы: не факт что норм результат (он не особо контролируется), в станзу надо подавать nlp(text), для больших данных это будет очень медленно.

In [20]:
import spacy
from spacy.matcher import Matcher
import re
nlp = spacy.load("en_core_web_sm")

In [22]:
tags = ['samsung galaxy', 'telephone', 'phone', 'smartphone', 'tablet', 'cellphone', 'model']

Для простого правила я буду брать прилагательные, потому что иначе будет совсем мусорно и неполезно.

In [48]:
matched_sents = []  # Collect data of matched sentences to be visualized
# 
pattern = [
           {"LENGTH": {"<=": 100}},
           {"LOWER": {"REGEX": "^samsung$"}, "OP": "*"},
           {"LOWER": {"REGEX": "^galaxy$"}},
           {"LOWER": {"REGEX": "(s.?(\d){0,4})|(note).?(\d){0,4}|(tab).?(\d){0,4}"}},
           {"LOWER": {"REGEX": "phone|smartphone|tablet|cellphone|model"}, "OP": "*"},
          ]

pattern2 = [{"POS": "ADJ"},
            {"LOWER": {"REGEX": "(^telephone$)|(^phone$)|(^smartphone$)|(^tablet$)|(^cellphone$)|(^model$)"}}                
           ]

matcher = Matcher(nlp.vocab)
matcher.add("samsung", [pattern])  # add pattern
matcher.add("phone", [pattern2])  # add pattern

for text in samsung_reviews2:
    flag = False
    for tag in tags:
        if tag in text:
            flag = True
            
    if flag == True:  
        doc = nlp(text)
        matches = matcher(doc)
        if len(matches) > 0:
            print([doc[start:end].text for match_id, start, end in matches][-1])
            matched_sents.append([doc[start:end].text for match_id, start, end in matches][-1])

newer model
replaytv model
earlier model
older model
same phone
latest model
right phone
expensive model
newer model
upconverting model
android phone
bd-5500c model
obsolete model
top model
next model
elite model
new model
same model
new model
comparible model
several telephone
different model
widescreen model
particular model
newest model
particular model
crt model
narrow model
older model
newer model
inch!>>>the model
new model
earlier model
newer model
good phone
numerous phone
previous model
pre-2004 model
8mp model
same model
correct model
same model
own model
particular model
new model
same model
older model
conventional model
newer model
new model
newer model
correct phone
later model
upper model
different model
later model
newer model
latest model
next model
same model
latest model
older model
new model
newer model
defective model
larger model
sharp model
full model
correct phone
120hz model
great model
bigger model
exact model
newer model
other model
later model
different mode

first tablet
the galaxy tab
little tablet
the galaxy tab
great tablet
other tablet
the galaxy tab
smart phone
of samsung galaxy tab
original galaxy tab
good tablet
my galaxy tab
second tablet
compact tablet
the galaxy tab
other tablet
small tablet
amazon.samsung galaxy tab
good tablet
good tablet
for samsung galaxy tab
of galaxy tabs
the galaxy tabs
good tablet
android tablet
all galaxy tab
the galaxy series
android phone
larger tablet
colored tablet
the galaxy's
the samsung galaxy tab
older tablet
the galaxy tab
the samsung galaxy tab
problemsamsung galaxy tab
next model
lighter model
particular model
big tablet
smart phone
sturdy tablet
nice tablet
real tablet
this samsung galaxy tab
galazy phone
great tablet
the samsung galaxy 8.9.pros
old tablet
inch galaxy devices
first tablet
the galaxy note
droid tablet
nice tablet
ipad tablet
the galaxy tab
protective tablet
nice tablet
smaller model
new model
fastest model
same model
my samsung galaxy tab
my samsung galaxy tab
my samsung galax

best tablet
nice tablet
new tablet
a galaxy tab
samsung phone
the galaxy tab
the samsung galaxy tab
ivso samsung galaxy tab
android phone
the galaxy tab
smart phone
the galaxy tab
perfect tablet
last tablet
gsm galaxy nexus
newer model
nice tablet
great tablet
the galaxy tab
the galaxy tab
the samsung galaxy tab
the galaxy tab
the galaxy tablet
nice tablet
the galaxy tab
first tablet
nice tablet
second tablet
right tablet
a samsung galaxy note
dumb phone
smaller phone
only tablet
fantastic tablet
my galaxy is
of samsung galaxy tab
great tablet
android phone
little tablet
chinese tablet
the galaxy tab
great tablet
the galaxy tab
the galaxy is
enough tablet
other tablet
the samsung galaxy tab
in galaxy tab
little tablet
with samsung galaxy s2
smart phone
the galaxy tab
this galaxy tab
sized tablet
acceptable tablet
ringbo samsung galaxy tab
small tablet
the samsung galaxy tab
small tablet
the galaxy tab
my galaxy note
bare tablet
our galaxy tablets
my samsung galaxy tab
new model
expensi

android tablet
the samsung galaxy tab
new tablet
have galaxy phones
second samsung galaxy tab
whole tablet
the galaxy tab
about galaxy tabs
smart phone
good tablet
first tablet
the samsung galaxy tab
memory).this tablet
vibrant phone
good tablet
original tablet
small tablet
nice tablet
best tablet
vibrant tablet
my samsung galaxy s
small tablet
a samsung galaxy s
all samsung galaxy tab2
good tablet
basic tablet
cool tablet
the galaxy tab
my samsung galaxy tab
first tablet
the galaxy tab
cellular phone
smart phone
first tablet
the galaxy tab
the samsung galaxy tab
other tablet
big tablet
this samsung galaxy tab
good tablet
portable tablet
a samsung galaxy s3
whole tablet
first tablet
little tablet
a samsung galaxy tab
the samsung galaxy tab
the galaxy tab
great tablet
good tablet
android tablet
affordable tablet
previous model
great tablet
great tablet
sized tablet
the galaxy tab
cool tablet
my galaxy note
the galaxy tablet
this samsung galaxy tab
my galaxy s3
great tablet
this galaxy t

have galaxy s4 phones
the samsung galaxy tab
nice tablet
right tablet
right tablet
fantastic tablet
disgruntled galaxy tab
my galaxy note
enabled model
android tablet
my galaxy tab
terrific tablet
awesome tablet
best tablet
android tablet
generic tablet
the samsung galaxy s5
the samsung galaxy tab
best tablet
right tablet
right tablet
the galaxy tab
the galaxy tab


In [44]:
tripple = []  # Collect data of matched sentences to be visualized
# 
pattern = [
           {"LOWER": {"REGEX": "^samsung$"}, "OP": "*"},
           {"LOWER": {"REGEX": "^galaxy$"}},
           {"LOWER": {"REGEX": "(s.?(\d){0,4})|(note).?(\d){0,4}|(tab).?(\d){0,4}"}},
           {"LOWER": {"REGEX": "phone|smartphone|tablet|cellphone|model"}, "OP": "*"},
          ]



matcher = Matcher(nlp.vocab)
matcher.add("samsung2", [pattern])  # add pattern

for text in samsung_reviews2:
    flag = False
    for tag in tags:
        if tag in text:
            flag = True
            
    if flag == True:  
        doc = nlp(text)
        matches = matcher(doc)
        if len(matches) > 0:
            print([doc[start:end].text for match_id, start, end in matches][-1])
            tripple.append([doc[start:end].text for match_id, start, end in matches][-1])

galaxy tab
galaxy tab
galaxy tablet
galaxy's
galaxy s phone
galaxy s
galaxy tab
galaxy tablet
galaxy s.if
galaxy tab
galaxy tab
galaxy's
galaxy tab
galaxy tab
galaxy tab
galaxy tab
galaxy tab
galaxy tab
galaxy tablet
galaxy tab
galaxy's
galaxy tab
galaxy tab
galaxy tab
galaxy service
galaxy as
galaxy tab
galaxy note
galaxy tab
galaxy tab
galaxy tablet
galaxy note
galaxy s3
galaxy tab
galaxy tab
galaxy note
galaxy tab
galaxy tablet
galaxy tab
galaxy tab
galaxy tab
galaxy tab
galaxy tab
galaxy tab
galaxy tab
galaxy tab
galaxy tab
galaxy tab
galaxy tab
galaxy tab
galaxy tab
galaxy tab
galaxy tablet
galaxy note
galaxy tab
galaxy tab
galaxy tab
galaxy tablet
galaxy tab
galaxy tablets
galaxy note
galaxy tab
galaxy s-10
galaxy tablets
galaxy tab
galaxy tablet
galaxy tab
galaxy tablet
galaxy tab
galaxy tab
galaxy tab
galaxy tab
galaxy tab
galaxy tab
galaxy tab
galaxy tab
galaxy tablet
galaxy tabs
galaxy tab
galaxy tab
galaxy tab
galaxy tab
galaxy tab
galaxy tab
galaxy tab
galaxy tab
galaxy tab

galaxy tab
galaxy tab
galaxy tab
galaxy s3
galaxy tab
galaxy tab
galaxy tab
galaxy tab
galaxy tablet
galaxy is
galaxy tab3
galaxy tab
galaxy note
galaxy tablet
galaxy tab
galaxy s3
galaxy tab
galaxy tab
galaxy tab
galaxy note
galaxy tab
galaxy tab
galaxy tab
galaxy s phone
galaxy tab
galaxy s3 phone
galaxy s
galaxy has
galaxy tab
galaxy tab
galaxy tab
galaxy tab
galaxy s4 phone
galaxy tab
galaxy tab
galaxy devices
galaxy tab
galaxy note
galaxy tab
galaxy tab
galaxy tab
galaxy tablet
galaxy note
galaxy tab
galaxy tab
galaxy is
galaxy tab
galaxy tab.3
galaxy tab
galaxy s3 phone
galaxy note
galaxy tab
galaxy just
galaxy tab
galaxy tab
galaxy doesn&#8217;t
galaxy s4
galaxy reads
galaxy tab
galaxy tab
galaxy is
galaxy tab
galaxy note
galaxy tab
galaxy tab
galaxy note
galaxy products
galaxy tab
galaxy tab
galaxy tab
galaxy tab
galaxy tab
galaxy tab
galaxy s4 phone
galaxy tab
galaxy tab
galaxy tab
galaxy tab
galaxy tab
galaxy tab
galaxy s4 phone
galaxy s4 phone
galaxy tab
galaxy tab
galaxy s4

Попытка склеить триграммы, но у меня вылетел юпитер и нет сил больше делать)

In [49]:
skleeno = {}
for t in tripple:
    skleeno[t] = '_'.join(t.split())

In [62]:
new_texts = []

for text in samsung_reviews2:
    for pat, rep in skleeno.items():
        try:
            text = re.sub(pat, rep, text)
        except:
            pass
    new_texts.append(text)

In [63]:
for text in new_texts:
    if 'galaxy_note' in text:
        print(text)

i ordered this because the galaxy_note 10.1 product says that it don't come in the box but it did so this was an extra.however i decided to keep it it is a great quality product the cord is the same length as what you get with your orginal tablet package.great delivery time and can't complain great product if you need an extra cord
as other reviewers have observed this unit didn't work on my galaxy_note 10.1 rather than charge it discharged i tried a variety of usb cables on this charger unit and the unit that came with my tablet the cables all worked with the original samsung charger unit none of them worked with this charger if you buy it make sure you test it asdap and return it if it doesn't work
the galaxy_note 10.1 requires quite a bit of charging power and this does that job well where as the charger for my galaxy_s3 doesn't.i just wish the cord to the tablet was longer i bought a 2nd charger and cable to use in the car.so far no problems i have an ac converter in my car so the 

Конец попытки

### Устарело (попытка сделать на простом матчинге)

In [ ]:
n_tags = ['samsung galaxy', 'samsung galaxy note', 'samsung galaxy tab']
tags = ['telephone', 'phone', 'smartphone', 'tablet']

In [ ]:
post_tags1 = 'tab|tablet|s'
post_tags2 = ['tab', 'tablet', 's\d']

In [ ]:
nes = []
for text in samsung_reviews:
    bigrams = list(nltk.bigrams(text.lower().split()))
    trigrams = list(nltk.trigrams(text.lower().split()))
    
#     for i, trigram in enumerate(trigrams):
#         trigram = ' '.join(trigram)
#         for tag in n_tags:
#             if tag == trigram:
#                 try:
#                     print(trigram[i-3], trigram, trigram[i+3])
#                 except IndexError:
#                     pass       
     
    tokens = text.lower().split()     
    
    for i, bigram in enumerate(bigrams):
        bigram = ' '.join(bigram)
        for tag in n_tags:
            if tag == bigram:
                try:
                    if re.search(post_tags1, tokens[i+2]):
                        print(tokens[i-1], bigram, tokens[i+4])
                    else:
                        print(tokens[i-1], bigram, tokens[i+2])
                except IndexError:
                    pass  


        
#     for i, token in enumerate(tokens):           
#         for tag in tags:
#             if token == tag:
#                 try:
#                     #print(text)
#                     print(' '.join(tokens[(i-2):(i+2)]))
#                     nes.append(' '.join(tokens[(i-2):(i+2)]))
#                     break
#                 except IndexError:
#                     pass

### Устарело кончилось

In [24]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

def normalize(text):
    # Take input text and return list of lemmas
    lemmas = []
    for t in tokenizer.tokenize(text):
        lemmas.append(lemmatizer.lemmatize(t))
    return lemmas

In [82]:
tok_texts = []
for text in samsung_reviews2:
    tok_texts.append(normalize(text))

In [66]:
import nltk
from nltk.collocations import *

# Коллекция метрик
bigram_measures = nltk.collocations.BigramAssocMeasures()
trigram_measures = nltk.collocations.TrigramAssocMeasures()

In [83]:
finder2 = BigramCollocationFinder.from_documents(tok_texts)
finder2.apply_freq_filter(2)

In [85]:
pmi_scores = finder2.score_ngrams(bigram_measures.pmi)

In [88]:
filtered_pmi_scores = []
for pair in tqdm(matched_sents):
    pair = tuple(pair.split())
    for pmi in pmi_scores:
        if pair == pmi[0]:
            filtered_pmi_scores.append(pmi)

100%|██████████████████████████████████████████████████████████████████████████████| 2143/2143 [01:57<00:00, 18.30it/s]


In [90]:
d_filtered_pmi_scores = {}

for pair in filtered_pmi_scores:
    d_filtered_pmi_scores['_'.join(pair[0])] = pair[1]

In [91]:
sorted(d_filtered_pmi_scores.items(), key=lambda x: x[1], reverse=True)

[('g2_phone', 9.439542312344807),
 ('d6400_model', 8.780555254193485),
 ('fancier_model', 8.102483349080847),
 ('htc_phone', 7.825433465964135),
 ('galaxy_note', 7.769953087203547),
 ('gingerbread_phone', 7.661934733681253),
 ('newer_model', 7.601789765257298),
 ('droid_phone', 7.202503115043957),
 ('particular_model', 7.199605815978952),
 ('convertible_tablet', 7.047534558109589),
 ('5mp_phone', 7.01807854390653),
 ('smart_phone', 6.951838973743705),
 ('mobile_phone', 6.928108847045614),
 ('international_model', 6.780555254193487),
 ('cellular_phone', 6.574471892430914),
 ('european_model', 6.458627159306122),
 ('android_phone', 6.410138365570258),
 ('previous_model', 6.338923544902929),
 ('mainstream_tablet', 6.3105689639433855),
 ('lesser_model', 6.261181095099907),
 ('older_model', 6.065260885124651),
 ('earlier_model', 5.979100933575566),
 ('newest_model', 5.959525395238806),
 ('android_tablet', 5.722512209937662),
 ('equivalent_model', 5.594688708882153),
 ('latest_model', 5.5789

In [92]:
dice_scores = finder2.score_ngrams(bigram_measures.dice)

filtered_dice_scores = []
for pair in tqdm(matched_sents):
    pair = tuple(pair.split())
    for dice in dice_scores:
        if pair == dice[0]:
            filtered_dice_scores.append(dice)

100%|██████████████████████████████████████████████████████████████████████████████| 2143/2143 [01:58<00:00, 18.03it/s]


In [93]:
d_filtered_dice_scores = {}

for pair in filtered_dice_scores:
    d_filtered_dice_scores['_'.join(pair[0])] = pair[1]

In [94]:
sorted(d_filtered_dice_scores.items(), key=lambda x: x[1], reverse=True)

[('galaxy_note', 0.15346938775510205),
 ('smart_phone', 0.06635071090047394),
 ('newer_model', 0.05082454717491214),
 ('android_phone', 0.04880478087649402),
 ('android_tablet', 0.04510786663761168),
 ('particular_model', 0.027226318774815655),
 ('older_model', 0.02702702702702703),
 ('previous_model', 0.022927219408157826),
 ('mobile_phone', 0.018737270875763747),
 ('same_model', 0.01826342671413094),
 ('different_model', 0.018204911092294666),
 ('other_tablet', 0.017666853617498597),
 ('new_model', 0.01652892561983471),
 ('great_tablet', 0.014835519243173511),
 ('expensive_model', 0.012281994595922378),
 ('other_model', 0.011857707509881422),
 ('best_tablet', 0.011626867006528933),
 ('latest_model', 0.011506849315068493),
 ('new_tablet', 0.011440107671601614),
 ('first_tablet', 0.011165387299371946),
 ('samsung_model', 0.010722224643682169),
 ('earlier_model', 0.008208736440926415),
 ('led_model', 0.007534533277521975),
 ('current_model', 0.007248396989127405),
 ('nice_tablet', 0.007

In [95]:
llh_scores = finder2.score_ngrams(bigram_measures.likelihood_ratio)

filtered_llh_scores = []
for pair in tqdm(matched_sents):
    pair = tuple(pair.split())
    for llh in llh_scores:
        if pair == llh[0]:
            filtered_llh_scores.append(llh)

100%|██████████████████████████████████████████████████████████████████████████████| 2143/2143 [01:59<00:00, 17.91it/s]


In [96]:
d_filtered_llh_scores = {}

for pair in filtered_llh_scores:
    d_filtered_llh_scores['_'.join(pair[0])] = pair[1]

In [99]:
final = sorted(d_filtered_llh_scores.items(), key=lambda x: x[1], reverse=True)
final

[('galaxy_note', 3422.409145993819),
 ('android_tablet', 1264.6858140798797),
 ('smart_phone', 981.4844933521193),
 ('newer_model', 823.8750613832055),
 ('android_phone', 686.7188721948897),
 ('particular_model', 391.0897885237406),
 ('older_model', 358.3497860497848),
 ('other_tablet', 323.83824007593836),
 ('previous_model', 296.67150497852754),
 ('samsung_model', 267.00728857351237),
 ('great_tablet', 260.4935696928853),
 ('same_model', 241.61111131811296),
 ('new_model', 232.02491150672302),
 ('different_model', 212.03779392952453),
 ('mobile_phone', 177.26755536068606),
 ('other_model', 163.97312055934344),
 ('best_tablet', 157.8468999690998),
 ('new_tablet', 150.54088288631024),
 ('first_tablet', 145.73282878094523),
 ('samsung_phone', 130.67826098329417),
 ('expensive_model', 123.2106181899652),
 ('latest_model', 122.31590804424611),
 ('earlier_model', 89.34090612141736),
 ('current_model', 66.90500385719162),
 ('smaller_tablet', 63.71095092764438),
 ('nice_tablet', 63.263092779

Последнее нравится больше всего

In [105]:
d = {}
for el in final:
    pair = el[0].split('_')
    try:
        d[pair[1]].append(pair[0])
    except:
        d[pair[1]] = []
        d[pair[1]].append(pair[0])

In [116]:
for key in d.keys():
    if key != 'note':
        
        print(key, ':\n', ', '.join(d[key][:5]))

tablet :
 android, other, great, best, new
phone :
 smart, android, mobile, samsung, htc
model :
 newer, particular, older, previous, samsung
smartphone :
 samsung, good
cellphone :
 samsung


Результат не совсем тот, который хотелось, но очень мусорный датасет.<br>
Задание на бонус - пункт 2 ниже.

Что можно сделать:
1. Склеить NE-многограмы в одну через подчеркивание, чтобы их тоже учитывать (не зря же я выделал такие правила)! Но мне кажется, даже так, они не будут в топах рейтинга (но я не уверен).
2. Брать больше информации из метатекста - например, категорию